In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
import pandas as pd
import numpy as np

# own Modules 
from models_sub_net_ls import LstmMse_LatentSpace, LstmMle_LatentSpace, AnalysisLayer
from models_mse import LstmMse
from models_mle import LstmMle_1
from data_preperator import DataPreperatorPrediction
from data_set import DataSet
from predictor import PredictorMse, PredictorMle, PredictorMseLatentSpaceAnalyser, PredictorMleLatentSpaceAnalyser

## Take care of these things before training:
- Select correct path and define droped_features
- Change parameter of model
- Change filed_location

## Parameters phm data

In [24]:
param = {
    "data" : {
        "path" : '../../data/phm_data_challenge/01_M01_DC_prediction_4.csv',
        "droped_feature" : ["stage", "Lot", "runnum", "recipe", "recipe_step",
                            "up time", "ongoing time", 
                            "ETCHSOURCEUSAGE", "ETCHAUXSOURCETIMER", 
                            "ETCHAUX2SOURCETIMER", "FIXTURESHUTTERPOSITION"
                           ],
        "features_not_to_scale": []
    },
    "model" : {
        "path" : "../../models/MLE_model/phm_dataFold xx_InputSize13_LayerLstm1_HiddenLstm15_HiddenFc75_Seq25.pt",
        "input_size" : 13,
        "n_hidden_lstm" : 15,
        "n_hidden_fc" : 75,
        "sequence_size" : 100,
        "batch_size" : 50,
        "lstm_layer" : 1,
        "dropout_rate_lstm": 0.0,
        "dropout_rate_fc": 0.2,
        "K":1
    },
        "anomaly_detection" : {
        "threshold_anomaly" : 0.3,
        "smooth_rate" : 0.05,
        "no_standard_deviation" : 2
    },
    "results": {
        "path" : "../visualisation/files/prediction/MLE/phm_4.csv",
    }
}

## Parameters artifical data

In [5]:
param = {
    "data" : {
        "path" : '../../data/artifical_signals/MLE_analysis/artifical_2_signals_errors.csv',
        "droped_feature" : ["anomaly"
                           ],
        "features_not_to_scale": []
    },
    "model" : {
        "path" : "../../models/MLE_model/artifical_2_signalsFold xx_InputSize2_LayerLstm1_HiddenLstm15_HiddenFc75_Seq25.pt",
        "input_size" : 2,
        "n_hidden_lstm" : 15,
        "n_hidden_fc" : 75,
        "sequence_size" : 100,
        "batch_size" : 50,
        "lstm_layer" : 1,
        "dropout_rate_lstm": 0.0,
        "dropout_rate_fc": 0.2,
        "K":1
    },
        "anomaly_detection" : {
        "threshold_anomaly" : 0.3,
        "smooth_rate" : 0.05,
        "no_standard_deviation" : 2
    },
    "results": {
        "path" : "../visualisation/files/prediction/MLE_LS/artifical_2_signals.csv",
    }
}

## Parameters cpps data

In [2]:
param = {
    "data" : {
        "path" : '../../data/cpps_data/cpps_data_predictive_maintenance.csv',
        "droped_feature" : ["status"
                           ],
        "features_not_to_scale": []
    },
    "model" : {
        "path" : "../../models/MLE_model/cpps_dataFold xx_InputSize10_LayerLstm1_HiddenLstm15_HiddenFc75_Seq25.pt",
        "input_size" : 10,
        "n_hidden_lstm" : 15,
        "n_hidden_fc" : 75,
        "sequence_size" : 100,
        "batch_size" : 50,
        "lstm_layer" : 1,
        "dropout_rate_lstm": 0.0,
        "dropout_rate_fc": 0.2,
        "K":1
    },
        "anomaly_detection" : {
        "threshold_anomaly" : 0.3,
        "smooth_rate" : 0.05,
        "no_standard_deviation" : 2
    },
    "results": {
        "path" : "../visualisation/files/prediction/MLE_LS/cpps_data.csv",
    }
}

## Standarize Data
First we have to apply normalisation to data. That is because the model works on the representation given by its input vectors. The scale of those numbers is part of the representation.
We should apply the exact same scaling as for training data. That means storing the scale and offset used with your training data, and using that again. <br>
__The mean and variance for each feature of the training data with which the model was trained (stake: 0.75):__

### Mean and Variance from phm Dataset
droped features="stage", "Lot", "runnum", "recipe", "recipe_step",
                            "up time", "ongoing time", 
                            "ETCHSOURCEUSAGE", "ETCHAUXSOURCETIMER", 
                            "ETCHAUX2SOURCETIMER", "FIXTURESHUTTERPOSITION"

In [25]:
mean_training_data =[ 0.21683119,  0.32121513,  0.31925213,  0.20097501,  0.45164471,  0.22914814,
  0.11604865,  0.27421592,  0.24393222, -0.13974937, -0.09739598, -0.07313758,
  0.18198089]
var_training_data =[0.75261122, 0.90482986, 0.91105327, 0.75504036, 1.07026701, 0.76708319,
 0.35172769, 0.83004988, 0.76964675, 0.57386915, 0.45912309, 0.2955709,
 1.61493449]

### Mean and Variance from artifical Dataset

In [6]:
mean_training_data= [-0.00393712, -0.01294209]
var_training_data = [49.18936568,  0.34270256]

### Mean and Variance form cpps Dataset

In [3]:
mean_training_data = [0.05330162, 0.03075699, -0.05636937, 0.0274802, 0.06536314, -0.04620979,-0.0745559, 
                      -0.08149049, -0.05318843, 0.11105582]
var_training_data = [0.02905961, 0.04473883, 0.05254194, 0.05198144, 0.07337494, 0.0666981, 0.07593811, 
                     0.0393896, 0.08028017, 0.0594492]

## Create DataLoader

In [26]:
data_preperator = DataPreperatorPrediction(path=param['data']['path'], 
                                           ignored_features = param["data"]["droped_feature"],
                                           mean_training_data=mean_training_data, 
                                           var_training_data=var_training_data, 
                                           first_order_difference=False 
                                          )                                  
preprocessed_data = data_preperator.prepare_data()
print(preprocessed_data.shape)

dataset = DataSet(preprocessed_data, 
                  timesteps=param["model"]["sequence_size"])
data_loader = DataLoader(dataset, 
                         batch_size=param['model']['batch_size'], 
                         num_workers=0, 
                         shuffle=False, 
                         drop_last=True)

(100000, 14)


In [27]:
for batch_idx, data in enumerate(data_loader):
    x,y = data
    print('Data of batch: {}'.format(batch_idx))
    print("Size of input data: {}".format(x.size()))
    print("Size of target data: {}".format(y.size()))
    if batch_idx >=1: break

Data of batch: 0
Size of input data: torch.Size([50, 100, 14])
Size of target data: torch.Size([50, 14])
Data of batch: 1
Size of input data: torch.Size([50, 100, 14])
Size of target data: torch.Size([50, 14])


## Define Model and load Parameters of trained model
### Model for MSE 

In [ ]:
model = LstmMse(batch_size=param['model']['batch_size'], 
                input_dim=param['model']['input_size'], 
                n_hidden_lstm=param['model']['n_hidden_lstm'], 
                n_hidden_fc=param['model']['n_hidden_fc'], 
                n_layers=param['model']['lstm_layer'], 
                dropout_rate_lstm= param['model']['dropout_rate_lstm'],
                dropout_rate_fc= param['model']['dropout_rate_fc']
                )

checkpoint = torch.load(param["model"]["path"])
model.load_state_dict(checkpoint['model_state_dict'])

### Model for MLE

In [28]:
model = LstmMle_1(batch_size=param['model']['batch_size'], 
                 input_dim=param['model']['input_size'], 
                 n_hidden_lstm=param['model']['n_hidden_lstm'], 
                 n_hidden_fc=param['model']['n_hidden_fc'], 
                 n_layers=param['model']['lstm_layer'], 
                 dropout_rate_lstm= param['model']['dropout_rate_lstm'],
                 dropout_rate_fc= param['model']['dropout_rate_fc'],
                 K = param['model']['K'])

checkpoint = torch.load(param["model"]["path"])
model.load_state_dict(checkpoint['model_state_dict'])

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

## Initialize Predictor
### Predictor for MSE Model

In [ ]:
predictor = PredictorMse(model=model,
                         path_data=param["data"]["path"],
                         columns_to_ignore=param["data"]["droped_feature"],
                         threshold_anomaly=param["anomaly_detection"]["threshold_anomaly"]
                         )

### Predictor for MLE Model

In [29]:
predictor = PredictorMle(model=model,
                         path_data=param["data"]["path"],
                         columns_to_ignore=param["data"]["droped_feature"],
                         threshold_anomaly=param["anomaly_detection"]["threshold_anomaly"],
                         no_standard_deviation=param["anomaly_detection"]["no_standard_deviation"]
                         )

## Predict

In [30]:
print("Start predicting.")
# Write header
with open(param["results"]["path"], "a+") as file:
            [file.write(column+";") for column in predictor.create_column_names_result()]
            file.write("\n")

for batch_number, (input_data, target_data) in enumerate(data_loader):
    # Predict sensor values in mini-batches
    batch_results = predictor.predict(input_data, target_data)
    
    # Write results to csv file
    with open(param["results"]["path"], "a") as file:
        for batch in batch_results:
            # Each result component of a singe prediction (ID, target, prediction, loss, latent space ...) is stored in lists
            # thus we have to unpack the list and seperate values with ;
            for value in batch:
                file.write(str(value)+";")
            file.write("\n")

    # Print status 
    if (batch_number*param['model']['batch_size'])%5000 == 0:
        print("Current status: " + str(param['model']['batch_size']*batch_number) + " samples are predicted.")

print("End of prediction.")

Start predicting.
Current status: 0 samples are predicted.
Current status: 5000 samples are predicted.
Current status: 10000 samples are predicted.
Current status: 15000 samples are predicted.
Current status: 20000 samples are predicted.
Current status: 25000 samples are predicted.
Current status: 30000 samples are predicted.
Current status: 35000 samples are predicted.
Current status: 40000 samples are predicted.
Current status: 45000 samples are predicted.
Current status: 50000 samples are predicted.
Current status: 55000 samples are predicted.
Current status: 60000 samples are predicted.
Current status: 65000 samples are predicted.
Current status: 70000 samples are predicted.
Current status: 75000 samples are predicted.
Current status: 80000 samples are predicted.
Current status: 85000 samples are predicted.
Current status: 90000 samples are predicted.
Current status: 95000 samples are predicted.
End of prediction.


## Tag anomalous samples

In [31]:
results_prediction = pd.read_csv(param["results"]["path"], sep=";")
# Values of column "loss" are exponentially smoothed and stored in a new column "smoothed loss"
# New column "anomaly" is created and sample is taged with 1 if anomalous behaviour (if smoothed loss is over threshold)
results = predictor.detect_anomaly(results_prediction, param["anomaly_detection"]["smooth_rate"])

In [32]:
results.head()

,ID,IONGAUGEPRESSURE target,ETCHBEAMVOLTAGE target,ETCHBEAMCURRENT target,ETCHSUPPRESSORVOLTAGE target,ETCHSUPPRESSORCURRENT target,FLOWCOOLFLOWRATE target,FLOWCOOLPRESSURE target,ETCHGASCHANNEL1READBACK target,ETCHPBNGASREADBACK target,...,Anomaly Sensor_4,Anomaly Sensor_5,Anomaly Sensor_6,Anomaly Sensor_7,Anomaly Sensor_8,Anomaly Sensor_9,Anomaly Sensor_10,Anomaly Sensor_11,Anomaly Sensor_12,Anomaly Sensor_13
0,300101.0,0.673644,0.804815,0.794134,0.341995,0.594840,0.524208,0.427323,0.630867,0.564256,...,0,0,0,1,0,0,0,0,0,0
1,300102.0,0.673644,0.804419,0.792009,0.341375,0.589761,0.524208,0.427323,0.630867,0.562412,...,0,0,0,1,0,0,0,0,0,0
2,300103.0,0.673644,0.804655,0.794628,0.340927,0.596423,0.524208,0.427323,0.630867,0.562412,...,0,0,0,1,0,0,0,0,0,0
3,300104.0,0.673644,0.804340,0.792788,0.341096,0.594522,0.519467,0.424578,0.632963,0.562412,...,0,0,0,1,0,0,0,0,0,0
4,300105.0,0.673644,0.804576,0.796753,0.341262,0.593887,0.519467,0.427323,0.632963,0.560713,...,0,0,0,1,0,0,0,0,0,0


## Combine prediction data with data which was not consider for inference

In [33]:
original_sensor_data = pd.read_csv(param["data"]["path"])
data_of_droped_feature = original_sensor_data.loc[:, param["data"]["droped_feature"]+["ID"]]
complete_data = results.merge(right=data_of_droped_feature, how="inner", on="ID")

## Save data to csv file

In [34]:
complete_data.to_csv(param["results"]["path"], sep=";", index=False)